Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic

In [5]:
batch_size = 128
l2_alfa = 0.003

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    l2 = l2_alfa * tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('Done.')            

Initialized
Minibatch loss at step 0: 27.505627
Minibatch accuracy: 7.8%
Validation accuracy: 12.8%
Test accuracy: 13.2%
Minibatch loss at step 500: 2.638179
Minibatch accuracy: 79.7%
Validation accuracy: 78.2%
Test accuracy: 85.1%
Minibatch loss at step 1000: 1.232659
Minibatch accuracy: 78.1%
Validation accuracy: 81.8%
Test accuracy: 88.1%
Minibatch loss at step 1500: 0.963136
Minibatch accuracy: 76.6%
Validation accuracy: 82.7%
Test accuracy: 88.8%
Minibatch loss at step 2000: 0.652613
Minibatch accuracy: 85.9%
Validation accuracy: 81.6%
Test accuracy: 88.2%
Minibatch loss at step 2500: 0.689939
Minibatch accuracy: 79.7%
Validation accuracy: 82.4%
Test accuracy: 89.0%
Minibatch loss at step 3000: 0.564014
Minibatch accuracy: 84.4%
Validation accuracy: 82.8%
Test accuracy: 89.0%
Done.


  * Bez L2 Test accuracy: 86.3%
  * Z 0.003 * L2 89%

## 1 hidden layer

In [8]:
batch_size = 128
hidden_count = 1024
l2_alfa = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    wh = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_count]))
    bh = tf.Variable(tf.zeros([hidden_count]))
    weights = tf.Variable(tf.truncated_normal([hidden_count, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, wh) + bh)
    logits = tf.matmul(h1, weights) + biases
    l2 = l2_alfa * (tf.nn.l2_loss(wh) + tf.nn.l2_loss(weights))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    model = tf.nn.softmax(logits)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, model], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            valid = session.run(model, feed_dict={tf_train_dataset: valid_dataset})
            print("Validation accuracy: %.1f%%" % accuracy(valid, valid_labels))
            test = session.run(model, feed_dict={tf_train_dataset: test_dataset})
            print("Test accuracy: %.1f%%" % accuracy(test, test_labels))
print('Done.')            

Initialized
Minibatch loss at step 0: 688.913452
Minibatch accuracy: 8.6%
Validation accuracy: 34.7%
Test accuracy: 36.8%
Minibatch loss at step 500: 199.977463
Minibatch accuracy: 71.9%
Validation accuracy: 78.0%
Test accuracy: 83.9%
Minibatch loss at step 1000: 115.556396
Minibatch accuracy: 75.8%
Validation accuracy: 81.5%
Test accuracy: 88.1%
Minibatch loss at step 1500: 69.374603
Minibatch accuracy: 79.7%
Validation accuracy: 82.3%
Test accuracy: 88.4%
Minibatch loss at step 2000: 41.268394
Minibatch accuracy: 85.9%
Validation accuracy: 85.6%
Test accuracy: 91.4%
Minibatch loss at step 2500: 25.373905
Minibatch accuracy: 82.8%
Validation accuracy: 86.8%
Test accuracy: 92.4%
Minibatch loss at step 3000: 15.318479
Minibatch accuracy: 89.8%
Validation accuracy: 87.5%
Test accuracy: 93.0%
Done.


  * Bez L2: Najlepszy wynik: 88.5%
  * l2_alfa = 0.001: 93.3%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [12]:
batch_size = 128
hidden_count = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    wh = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_count]))
    bh = tf.Variable(tf.zeros([hidden_count]))
    weights = tf.Variable(tf.truncated_normal([hidden_count, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, wh) + bh)
    logits = tf.matmul(h1, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    model = tf.nn.softmax(logits)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (20 * batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, model], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            valid = session.run(model, feed_dict={tf_train_dataset: valid_dataset})
            print("Validation accuracy: %.1f%%" % accuracy(valid, valid_labels))
            test = session.run(model, feed_dict={tf_train_dataset: test_dataset})
            print("Test accuracy: %.1f%%" % accuracy(test, test_labels))    
print('Done.')

Initialized
Minibatch loss at step 0: 294.313049
Minibatch accuracy: 7.8%
Validation accuracy: 39.7%
Test accuracy: 42.2%
Minibatch loss at step 500: 0.160908
Minibatch accuracy: 99.2%
Validation accuracy: 80.5%
Test accuracy: 86.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Test accuracy: 86.9%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Test accuracy: 86.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Test accuracy: 86.9%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Test accuracy: 86.9%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Test accuracy: 86.9%
Done.


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### Dla mniejszego zbioru danych

In [13]:
batch_size = 128
hidden_count = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_keep_prob = tf.placeholder("float")
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    wh = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_count]))
    bh = tf.Variable(tf.zeros([hidden_count]))
    weights = tf.Variable(tf.truncated_normal([hidden_count, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, wh) + bh)
    h1_drop = tf.nn.dropout(h1, tf_keep_prob)
    logits = tf.matmul(h1_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    model = tf.nn.softmax(logits)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (20 * batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob: 0.5}
        _, l, predictions = session.run([optimizer, loss, model], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            valid = session.run(model, feed_dict={tf_train_dataset: valid_dataset, tf_keep_prob: 1})
            print("Validation accuracy: %.1f%%" % accuracy(valid, valid_labels))
            test = session.run(model, feed_dict={tf_train_dataset: test_dataset, tf_keep_prob: 1})
            print("Test accuracy: %.1f%%" % accuracy(test, test_labels))    
print('Done.')

Initialized
Minibatch loss at step 0: 493.836914
Minibatch accuracy: 9.4%
Validation accuracy: 32.3%
Test accuracy: 33.7%
Minibatch loss at step 500: 11.829367
Minibatch accuracy: 94.5%
Validation accuracy: 81.3%
Test accuracy: 87.9%
Minibatch loss at step 1000: 1.289527
Minibatch accuracy: 95.3%
Validation accuracy: 81.7%
Test accuracy: 88.3%
Minibatch loss at step 1500: 0.976872
Minibatch accuracy: 97.7%
Validation accuracy: 81.9%
Test accuracy: 88.5%
Minibatch loss at step 2000: 0.000001
Minibatch accuracy: 100.0%
Validation accuracy: 82.0%
Test accuracy: 88.7%
Minibatch loss at step 2500: 0.000052
Minibatch accuracy: 100.0%
Validation accuracy: 82.3%
Test accuracy: 88.4%
Minibatch loss at step 3000: 0.013458
Minibatch accuracy: 99.2%
Validation accuracy: 82.0%
Test accuracy: 88.7%
Done.


Best result with dropout: 88.7% (without 86.9%)

### Dla pełnego zbioru danych

In [16]:
batch_size = 128
hidden_count = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_keep_prob = tf.placeholder("float")
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    wh = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_count]))
    bh = tf.Variable(tf.zeros([hidden_count]))
    weights = tf.Variable(tf.truncated_normal([hidden_count, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, wh) + bh)
    h1_drop = tf.nn.dropout(h1, tf_keep_prob)
    logits = tf.matmul(h1_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    model = tf.nn.softmax(logits)
    
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob: 0.5}
        _, l, predictions = session.run([optimizer, loss, model], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            valid = session.run(model, feed_dict={tf_train_dataset: valid_dataset, tf_keep_prob: 1})
            print("Validation accuracy: %.1f%%" % accuracy(valid, valid_labels))
            test = session.run(model, feed_dict={tf_train_dataset: test_dataset, tf_keep_prob: 1})
            print("Test accuracy: %.1f%%" % accuracy(test, test_labels))    
print('Done.')

Initialized
Minibatch loss at step 0: 500.403381
Minibatch accuracy: 7.8%
Validation accuracy: 28.8%
Test accuracy: 31.0%
Minibatch loss at step 1000: 14.514444
Minibatch accuracy: 73.4%
Validation accuracy: 81.3%
Test accuracy: 87.7%
Minibatch loss at step 2000: 6.388087
Minibatch accuracy: 73.4%
Validation accuracy: 80.5%
Test accuracy: 87.2%
Minibatch loss at step 3000: 3.866124
Minibatch accuracy: 76.6%
Validation accuracy: 79.9%
Test accuracy: 85.8%
Minibatch loss at step 4000: 10.567472
Minibatch accuracy: 71.9%
Validation accuracy: 80.8%
Test accuracy: 87.2%
Minibatch loss at step 5000: 2.317041
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Test accuracy: 87.6%
Minibatch loss at step 6000: 1.574266
Minibatch accuracy: 78.1%
Validation accuracy: 82.0%
Test accuracy: 88.3%
Minibatch loss at step 7000: 1.330148
Minibatch accuracy: 75.0%
Validation accuracy: 81.9%
Test accuracy: 88.4%
Minibatch loss at step 8000: 2.262365
Minibatch accuracy: 76.6%
Validation accuracy: 81.8%
T

Best result with 1000 epoch is 89%

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
batch_size = 128
hidden_count1 = 1024
hidden_count2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_keep_prob = tf.placeholder("float")
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    wh1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_count1]))
    bh1 = tf.Variable(tf.zeros([hidden_count1]))
    wh2 = tf.Variable(tf.truncated_normal([hidden_count1, hidden_count2]))
    bh2 = tf.Variable(tf.zeros([hidden_count2]))
    weights = tf.Variable(tf.truncated_normal([hidden_count2, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    h1 = tf.nn.relu6(tf.matmul(tf_train_dataset, wh1) + bh1)
    h1_drop = tf.nn.dropout(h1, tf_keep_prob)
    h2 = tf.nn.relu6(tf.matmul(h1_drop, wh2) + bh2)
    h2_drop = tf.nn.dropout(h2, tf_keep_prob)
    logits = tf.matmul(h2_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # Predictions for the training, validation, and test data.
    model = tf.nn.softmax(logits)
    
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob: 0.5}
        _, l, predictions = session.run([optimizer, loss, model], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            valid = session.run(model, feed_dict={tf_train_dataset: valid_dataset, tf_keep_prob: 1})
            print("Validation accuracy: %.1f%%" % accuracy(valid, valid_labels))
            test = session.run(model, feed_dict={tf_train_dataset: test_dataset, tf_keep_prob: 1})
            print("Test accuracy: %.1f%%" % accuracy(test, test_labels))    
print('Done.')

Initialized
Minibatch loss at step 0: 192.071411
Minibatch accuracy: 8.6%
Validation accuracy: 11.3%
Test accuracy: 11.6%
Minibatch loss at step 1000: 4.244045
Minibatch accuracy: 48.4%
Validation accuracy: 48.0%
Test accuracy: 52.9%
Minibatch loss at step 2000: 3.552507
Minibatch accuracy: 44.5%
Validation accuracy: 53.0%
Test accuracy: 57.5%
Minibatch loss at step 3000: 2.868275
Minibatch accuracy: 57.0%
Validation accuracy: 61.0%
Test accuracy: 66.7%
Minibatch loss at step 4000: 2.727026
Minibatch accuracy: 50.8%
Validation accuracy: 62.4%
Test accuracy: 68.7%
Minibatch loss at step 5000: 2.396410
Minibatch accuracy: 52.3%
Validation accuracy: 59.6%
Test accuracy: 65.8%
Minibatch loss at step 6000: 2.812232
Minibatch accuracy: 53.1%
Validation accuracy: 64.9%
Test accuracy: 72.3%
Minibatch loss at step 7000: 1.542667
Minibatch accuracy: 64.1%
Validation accuracy: 69.6%
Test accuracy: 76.8%
Minibatch loss at step 8000: 1.799317
Minibatch accuracy: 60.9%
Validation accuracy: 66.6%
Tes

Najlepszy wynik 76%